In [ ]:
#导入oled屏幕库 Import oled screen library
import sys
sys.path.append('/home/pi/software/oled_yahboom/')
from yahboom_oled import *
# 创建oled对象 Create an oled object
oled = Yahboom_OLED(debug=False)
#bgr8转jpeg格式 bgr8 to jpeg format
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# 导入库并显示摄像头显示组件 Import the library and display the camera display component

In [ ]:
import cv2
import numpy as np
import pyzbar.pyzbar as pyzbar
from PIL import Image
import ipywidgets.widgets as widgets

# from picamera2 import Picamera2, Preview
# import libcamera

image_widget = widgets.Image(format='jpeg', width=640, height=480)
display(image_widget)                                      #显示摄像头组件 Display camera components

# 定义解析二维码接口 Define the QR code parsing interface

In [ ]:
def decodeDisplay(image):
    barcodes = pyzbar.decode(image)
    qrtype_str=f'[INFO] Found None'
    qrcode_str=f'barcode: None'
    for barcode in barcodes:
        # 提取二维码的边界框的位置 Extract the position of the bounding box of the QR code
        # 画出图像中条形码的边界框 Draw the bounding box of the barcode in the image
        (x, y, w, h) = barcode.rect
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 225, 225), 2)

        # 提取二维码数据为字节对象，所以如果我们想在输出图像上 Extract the QR code data as a bytes object, so if we want to output it on the image
        # 画出来，就需要先将它转换成字符串 To draw it, you need to convert it into a string first
        datatemp=0
        barcodeData = barcode.data.decode("utf-8")
        barcodeType = barcode.type
        
        # 绘出图像上条形码的数据和条形码类型 Draw the barcode data and barcode type on the image
        text = "{} ({})".format(barcodeData, barcodeType)
        cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 225, 225), 2)

        # 向终端打印条形码数据和条形码类型 Print barcode data and barcode type to the terminal
        print("[INFO] Found {} barcode: {}".format(barcodeType, barcodeData))

        qrtype_str=f'[INFO] Found {barcodeType}'
        qrcode_str=f'barcode:{barcodeData}'

    oled.clear()
    oled.add_line(qrtype_str, 1)
    oled.add_line(qrcode_str, 3)
    oled.refresh()

    return image

def detect():
    oled.init_oled_process() #初始化oled进程 Initialize oled process
    camera = cv2.VideoCapture(0) #打开摄像头/dev/video0
    camera.set(3, 320)
    camera.set(4, 240)
    camera.set(5, 30)  #设置帧率 Setting the frame rate
    # fourcc = cv2.VideoWriter_fourcc(*"MPEG")
    # camera.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
    # camera.set(cv2.CAP_PROP_BRIGHTNESS, 40) #设置亮度 -64 - 64  0.0 Set Brightness -64 - 64 0.0
    # camera.set(cv2.CAP_PROP_CONTRAST, 50) #设置对比度 -64 - 64  2.0 Set Contrast -64 - 64 2.0
    # camera.set(cv2.CAP_PROP_EXPOSURE, 156) #设置曝光值 1.0 - 5000  156.0 Set the exposure value 1.0 - 5000 156.0
    ret, frame = camera.read()

    # picam2 = Picamera2()  
    # camera_config = picam2.create_preview_configuration(main={"format":'RGB888',"size":(320,240)})
    # camera_config["transform"] = libcamera.Transform(hflip=1, vflip=1)
    # picam2.configure(camera_config)
    # picam2.start() 
    # frame = picam2.capture_array()
    
    image_widget.value = bgr8_to_jpeg(frame)
    try:
        while True:
            # 读取当前帧 Read the current frame
            ret, frame = camera.read()
            #frame = picam2.capture_array()
            # 转为灰度图像 Convert to grayscale image
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            im = decodeDisplay(gray)

            cv2.waitKey(5)
            image_widget.value = bgr8_to_jpeg(im)
            # 如果按键q则跳出本次循环 If you press the q key, you will exit this loop.
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    except:
        #picam2.stop()
        camera.release()
        # 恢复屏幕基础数据显示 Restore basic data display on screen
        os.system("python3 /home/pi/software/oled_yahboom/yahboom_oled.py &")

# 主进程 Main Process

In [ ]:
detect()

In [ ]:
#最后需要释放掉摄像头的占用 Finally, you need to release the camera's occupancy
camera.release()
oled.init_oled_process() #初始化oled进程 Initialize oled process
# 恢复屏幕基础数据显示 Restore basic data display on screen
os.system("python3 /home/pi/software/oled_yahboom/yahboom_oled.py &")
# picam2.stop()
# picam2.close()
#最后需要释放掉摄像头的占用 Finally, you need to release the camera's occupancy